In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import requests
import json
import time
import pandas as pd

from tqdm import tqdm

riot_api_key = '' ##본인이 발급한 riot developer api key 입력
region = 'kr'

In [ ]:
challenger_url = f'https://{region}.api.riotgames.com/tft/league/v1/challenger?api_key={riot_api_key}'
response = requests.get(challenger_url)
challenger_data = response.json()
c_puuids = []

try:
  for entry in challenger_data['entries']:
      summoner_id = entry['summonerId']
      summoner_url = f'https://{region}.api.riotgames.com/tft/summoner/v1/summoners/{summoner_id}?api_key={riot_api_key}'
      summoner_response = requests.get(summoner_url)
      summoner_data = summoner_response.json()
      try:
          puuid = summoner_data['puuid']
          c_puuids.append(puuid)
      except KeyError:
          continue
except KeyError:
  pass

In [ ]:
grandmaster_url = f'https://{region}.api.riotgames.com/tft/league/v1/grandmaster?api_key={riot_api_key}'
response = requests.get(grandmaster_url)
grandmaster_data = response.json()
g_puuids = []

print(grandmaster_data)

try:
    for entry in grandmaster_data['entries']:
        summoner_id = entry['summonerId']
        summoner_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={riot_api_key}'
        summoner_response = requests.get(summoner_url)
        summoner_data = summoner_response.json()
        try:
            puuid = summoner_data['puuid']
            g_puuids.append(puuid)
        except KeyError:
            continue
except KeyError:
    pass

{'tier': 'GRANDMASTER', 'leagueId': 'f1e46b7f-5703-3dad-b9c5-6a8f5dfa5598', 'queue': 'RANKED_TFT', 'name': "Soraka's Enforcers", 'entries': [{'summonerId': '7andAUIsRWH3aNZO4fu5zsPoineCc6Z8ukaTtjnz2NH-U9c', 'leaguePoints': 637, 'rank': 'I', 'wins': 285, 'losses': 231, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}, {'summonerId': 'ln15fpx50L0Q-eegomVksMh1hJ_wdXK_Ifvf3DNsHAZHpTlNGhu2N5vXwA', 'leaguePoints': 691, 'rank': 'I', 'wins': 213, 'losses': 174, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}, {'summonerId': 'tQiSskQsAcq_FBr_JtMyBNnxjkH84Sus8eRx29SpS2IxztpX-s1ZpN9Xtw', 'leaguePoints': 644, 'rank': 'I', 'wins': 249, 'losses': 227, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}, {'summonerId': 'muOuPUmpO5kSqH5MalJ-ZX0YmXIgUVBEsp1-f4xtr6BGNxVgBh1Dw-mRFw', 'leaguePoints': 713, 'rank': 'I', 'wins': 130, 'losses': 77, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}, 

In [ ]:
# challenger_match_ids = []
# for puuid in tqdm(c_puuids):
#     match_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={riot_api_key}'
#     match_response = requests.get(match_url)
#     matches = match_response.json()
#     challenger_match_ids.extend(matches)
#     time.sleep(1.2)

grandmaster_match_ids = []
for puuid in tqdm(g_puuids):
    match_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={riot_api_key}'
    match_response = requests.get(match_url)
    matches = match_response.json()
    grandmaster_match_ids.extend(matches)
    time.sleep(1.2)  # API rate limit을 피하기 위해 딜레이 추가

100%|██████████| 591/591 [23:37<00:00,  2.40s/it]


In [ ]:
#print(len(challenger_match_ids))
print(len(grandmaster_match_ids))

11820


In [ ]:
# challenger_match_infos = []
# for match_id in tqdm(challenger_match_ids):
#   match_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}?api_key={riot_api_key}'
#   match_response = requests.get(match_url)
#   match_data = match_response.json()
#   print(match_data)
#   challenger_match_infos.append(match_data)
#   time.sleep(1.2)

In [ ]:
output_file_path = 'challenger_match_infos.json'
with open(output_file_path, 'w') as json_file:
    json.dump(challenger_match_infos, json_file)

print(f"Data saved to {output_file_path}")

Data saved to challenger_match_infos.json


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

grandmaster_match_infos = []

def fetch_match_data(match_id):
    match_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}?api_key={riot_api_key}'
    response = requests.get(match_url)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 429:
        print(f"Rate limit exceeded. Sleeping for 2 minutes.")
        time.sleep(120)
        return fetch_match_data(match_id)
    else:
        print(f"Error {response.status_code} for match_id {match_id}")
        return None

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(fetch_match_data, match_id) for match_id in grandmaster_match_ids]
    for future in tqdm(as_completed(futures), total=len(grandmaster_match_ids)):
        match_data = future.result()
        if match_data is not None:
            grandmaster_match_infos.append(match_data)
        time.sleep(1.2)

output_file_path = '/content/drive/MyDrive/grandmaster_match_infos.json'
with open(output_file_path, 'w') as json_file:
    json.dump(grandmaster_match_infos, json_file)

print(f"Data saved to {output_file_path}")

  0%|          | 0/11820 [00:00<?, ?it/s]

Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.
Rate limit exceeded. Sleeping for 2 minutes.


In [ ]:
output_file_path = '/content/drive/MyDrive/dataexp_final/grandmaster_match_infos_3.json'
with open(output_file_path, 'w') as json_file:
    json.dump(grandmaster_match_infos, json_file)

print(f"Data saved to {output_file_path}")